## 1 Define the Model URI

In [0]:

# XGBoost
run_id = "c8a36d6f9bba4e42b4da7d60135d56d9"
model_uri = f"runs:/{run_id}/model_xgboost" 


## 2 Register the Model in the MLflow Model Registry

In [0]:
import mlflow

model_name = "ckd_xgboost_model"

# Register the model
model_name = "ckd_xgboost_model"

# Register the model
result = mlflow.register_model(model_uri=model_uri, name=model_name)
print(f"Registered Model Name: {result.name}, Version: {result.version}")


## 3 Load the Registered Model for Inference 
We can now load it from the registry like this:

In [0]:
import pandas as pd
from mlflow.pyfunc import load_model

# Load model from registry (latest version)
model = load_model(f"models:/{model_name}/latest")

# Make predictions
predictions = model.predict(X_test)
print(predictions[:5])


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = "/dbfs/FileStore/tables/kidney_disease.csv"
df = pd.read_csv(file_path)

# Preprocess the dataset
df.replace("?", pd.NA, inplace=True)
for col in ['age', 'bp', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna()

# Encode the target column
df['classification'] = df['classification'].map({'ckd': 1, 'notckd': 0})

# Feature + target split
X = df.drop(columns=['classification'])
y = df['classification']

# Dummy encoding (if needed for categorical features)
X = pd.get_dummies(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
# Drop 'id' column if it's not part of the model features
X_test = X_test.drop(columns=['id'])

# Make predictions
predictions = model.predict(X_test)
print(predictions[:5])
X = df.drop(columns=['classification', 'id'])
# feature names match 

In [0]:
from sklearn.metrics import roc_auc_score

# Predict probabilities
probs = model.predict(X_test)
auc_score = roc_auc_score(y_test, probs)

print("AUC Score:", round(auc_score, 4))
